# Chapter 8, dictionaries I guess?

The chapter is actually about tuples and pairs, important thing is that in OCaml _lists_ are separated with semicolon `;` while _tuple pairs_ are separated by colon `,`.

Tuple pairs can have different types, for example `(1, 'a')` and different than list it cannot be ever _empty_.

The type representation of a _pair_ is `'a * 'b` notice the `*` in the middle, so the previous tuple `(1, 'a')` would be represented as `int * char`.

Because tuples has only _one representation_ you can decompose them easily:

In [3]:
let coordinates = (1, 2)
let x, y = coordinates

val coordinates : int * int = (1, 2)


val x : int = 1
val y : int = 2


In this chapter we implement our own dictionary structure. It is easy, we say that a dictionary is a list of key/value tuples, so a dictionary is `('a * 'b) list`

In [5]:
let dict = [(1,4); (2,8); (3,12)]

val dict : (int * int) list = [(1, 4); (2, 8); (3, 12)]


We need a method to look a value in the dictionary by its key:

In [10]:
(* We don't need an inner loop here, the function is tail recursive by nature *)

let rec lookup key d =
  match d with
  | hd :: tl ->
    let k, v = hd in
    if key = k then v else lookup key tl
  | _ -> raise (Not_found)
;;

lookup 2 [(1,4); (2,8); (3,12)] ;;
lookup 10 [(1,4); (2,8); (3,12)]

val lookup : 'a -> ('a * 'b) list -> 'b = <fun>


- : int = 8


error: runtime_error

We can create a function to check a key exists in the dictionary:

In [12]:
let key_exists k d =
  try let _ = lookup k d in true with (* Notice how we say 'if success then return true' *)
  | Not_found -> false
;;

val key_exists : 'a -> ('a * 'b) list -> bool = <fun>


## Exercises

 * Write a function to determine the number of different keys in a dictionary.
 
_This question doesn't make sense, because keys are unique it will be enough to do `length d`_

 * Define a function `replace` which is like `add`, but raises `Not_found` if the key is not already there.

In [35]:
let replace k v d =
  let rec loop l acc =
    match l with
    | (key, value) :: tl ->
      if key = k then loop tl ((k, v) :: acc)
      else loop tl ((key, value) :: acc)
    | _ -> acc in
  if key_exists k d then loop (List.rev d) [] else raise (Not_found)
;;

replace 2 2 [(1,5);(2,10);(3,15)];;

val replace : 'a -> 'b -> ('a * 'b) list -> ('a * 'b) list = <fun>


- : (int * int) list = [(1, 5); (2, 2); (3, 15)]


 * Write a function to build a dictionary from two equal length lists, one containing keys and another containing values. Raise the exception Invalid_argument if the lists are not of equal length.

In [41]:
let zip a b =
  let rec loop a' b' acc =
    match a', b' with
    | ha :: ta, hb :: tb ->
      loop ta tb ((ha, hb) :: acc)
    | _, _ -> acc in
  if List.length a <> List.length b then raise (Invalid_argument "zip")
  else loop (List.rev a) (List.rev b) []
;;

zip [1;2;3] [2;4;6] ;;
zip [1;2;3] [2;4;6;8]

val zip : 'a list -> 'b list -> ('a * 'b) list = <fun>


- : (int * int) list = [(1, 2); (2, 4); (3, 6)]


error: runtime_error

 * Now write the inverse function: given a dictionary, return the pair of two lists – the first containing all the keys, and the second containing all the values.

In [44]:
let unfold d =
  let rec loop l acc =
    match l with
    | (x, y) :: tl -> 
      let a, b = acc in
      loop tl ((x :: a), (y :: b))
    | _ -> acc in
  loop (List.rev d) ([], [])
;;

let keys, values = unfold [(1, 2); (2, 4); (3, 6)]

val unfold : ('a * 'b) list -> 'a list * 'b list = <fun>


val keys : int list = [1; 2; 3]
val values : int list = [2; 4; 6]


 * Define a function to turn any list of pairs into a dictionary. If duplicate keys are found, the value associated with the first occurrence of the key should be kept.

In [52]:
let make_dict l =
  let rec loop l' acc =
    match l' with
    | (k, v) :: tl ->
      if key_exists k acc then loop tl acc
      else loop tl ((k, v) :: acc)
    | _ -> acc in
  List.rev (loop l []) (* There should be a way to get rid of that rev! *)
;;

make_dict [(1, 2); (2, 4); (3, 6); (2,10)]

val make_dict : ('a * 'b) list -> ('a * 'b) list = <fun>


- : (int * int) list = [(1, 2); (2, 4); (3, 6)]


 * Write the function `union a b` which forms the union of two dictionaries. The union of two dictionaries is the dictionary containing all the entries in one or other or both. In the case that a key is contained in both dictionaries, the value in the first should be preferred.

In [63]:
let union a b =
  let not x = if x = true then false else true in
  let not_in x = let k, v = x in not (key_exists k a) in
  let b' = List.filter not_in b in
  a @ b'
;;

union [(1,2);(3,4);(5,6)] [(7,8);(9,0);(3,15)]

val union : ('a * 'b) list -> ('a * 'b) list -> ('a * 'b) list = <fun>


- : (int * int) list = [(1, 2); (3, 4); (5, 6); (7, 8); (9, 0)]
